## Install Python dependencies

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 191.4 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 329.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 300.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 217.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 266.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 223.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 241.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 251.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 311.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 276.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 306.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 285.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

Import the dependencies for the model training code:

In [2]:
import numpy as np
import pandas as pd
import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tf2onnx
import onnx
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

2024-09-23 06:48:19.334470: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 06:48:19.682976: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-23 06:48:19.947570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 06:48:20.445931: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 06:48:20.556748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 06:48:21.047714: I tensorflow/core/platform/cpu_feature_gu

In [3]:
#Laoding the dataset
Data = pd.read_csv('caloric_exp.csv')
Data.head()

,user_id,date,day,age,gender,weight,height_in_cms,digestive_disorder,type_of_workout,additional_activity,...,followed_diet_plan,taking_supplements,fitness_goal,target_weight,target_time_in_months,actual_calorie_intake_in_kcal_per_day,actual_carb_intake_in_gms,actual_protein_intake_in_gms,actual_fat_intake_in_gms,recommended_calorie_intake_in_kcal_per_day
0,1002,2022-01-03,Monday,37,Male,86,154,No,Light Weight Training,Zumba,...,No,No,Weight Loss,6,2,2370,296,118,79,2296
1,1002,2022-01-04,Tuesday,37,Male,86,154,No,Calesthenics,Zumba,...,No,No,Weight Loss,6,2,1965,245,98,65,1906
2,1002,2022-01-05,Wednesday,37,Male,86,154,No,Calesthenics,Zumba,...,Yes,No,Weight Loss,6,2,1695,211,84,56,1848
3,1002,2022-01-06,Thursday,37,Male,86,154,No,Rest,NaN,...,Yes,No,Weight Loss,6,2,1956,244,97,65,2136
4,1002,2022-01-07,Friday,37,Male,86,154,No,Light Weight Training,Zumba,...,No,No,Weight Loss,6,2,2141,267,107,71,2083


In [4]:
# Set the input (X) and output (Y) data.

X = Data.drop(columns = ['user_id','date','recommended_calorie_intake_in_kcal_per_day'])
y = Data['recommended_calorie_intake_in_kcal_per_day']


In [5]:
# Split the data into training and testing sets so you have something to test the trained model with.
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2)

In [6]:
Path("artifact").mkdir(parents=True, exist_ok=True)
with open("artifact/test_data.pkl", "wb") as handle:
    pickle.dump((X_test, y_test), handle)

In [7]:
#fetching the cols:
num_cols = X_train.select_dtypes(include='number').columns.to_list()
cat_cols = X_train.select_dtypes(exclude='number').columns.to_list()

In [8]:
#encoding the categorical_variables:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False)

In [9]:
#importing the scaler
scaler = StandardScaler()

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

ct = ColumnTransformer(
    transformers = [('Encoder',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),cat_cols),
                    ('Scaler',StandardScaler(),num_cols)],
                    remainder='passthrough')


In [11]:
ct_01= ct.fit(X_train)

In [12]:
with open("artifact/column_transformer.pkl", "wb") as handle:
    pickle.dump(ct_01, handle)

## Build the model

The model is a simple, fully-connected, deep neural network, containing three hidden layers and one output layer.

In [13]:
X_train_new = ct.fit_transform(X_train)

In [14]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = X_train_new.shape[1]))
# model.add(Dropout(0.2))
# model.add(Dense(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Dense(1, activation = 'linear'))
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['r2_score'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,185 (4.63 KB)

 Non-trainable params: 0 (0.00 B)

## Train the model

Training a model is often the most time-consuming part of the machine learning process.  Large models can take multiple GPUs for days.  Expect the training on CPU for this very simple model to take a minute or more.

In [15]:
# Train the model and get performance
import os

epochs = 200
history = model.fit(X_train_new, y_train.values, epochs=epochs, \
                    validation_data=(ct.transform(X_val),y_val), \
                    verbose = True)
print("Training of model is complete")

Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5400639.5000 - r2_score: -42.7886 - val_loss: 5304363.5000 - val_r2_score: -43.6995
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5280788.0000 - r2_score: -41.9115 - val_loss: 5105259.5000 - val_r2_score: -42.0217
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5041993.0000 - r2_score: -38.9881 - val_loss: 4794398.5000 - val_r2_score: -39.4021
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4727885.0000 - r2_score: -38.9489 - val_loss: 4397865.5000 - val_r2_score: -36.0605
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4283475.0000 - r2_score: -34.4915 - val_loss: 3927512.7500 - val_r2_score: -32.0969
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3823060.0000 - r2_score: -31.3378 - val_loss: 3420021.7500 - val_r2_score: -27.8203
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3323750.2500 - r2_score: -25.7750 - val_loss: 2907306.5000 - val_r2_s

## Save the model file

In [16]:
os.makedirs("models/caloric/1", exist_ok=True)

In [17]:
saved_model_path = "models/caloric/1/my_model"
model.export(saved_model_path)

INFO:tensorflow:Assets written to: models/caloric/1/my_model/assets


INFO:tensorflow:Assets written to: models/caloric/1/my_model/assets


Saved artifact at 'models/caloric/1/my_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 35), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139875524700816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139875524699056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139875524897952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139875524897776: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [27]:
X_test.dropna().sample(5).to_csv('X_test.csv',index = False)

In [46]:
X_test.columns

Index(['day', 'age', 'gender', 'weight', 'height_in_cms', 'digestive_disorder',
       'type_of_workout', 'additional_activity', 'total_calories_burned',
       'body_temperature', 'workout_intensity', 'followed_diet_plan',
       'taking_supplements', 'fitness_goal', 'target_weight',
       'target_time_in_months', 'actual_calorie_intake_in_kcal_per_day',
       'actual_carb_intake_in_gms', 'actual_protein_intake_in_gms',
       'actual_fat_intake_in_gms'],
      dtype='object')

In [ ]:
X_test.iloc[0,:].values

In [28]:
!pip install tf2onnx


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [39]:
#!mo --saved_model_dir models/caloric/1/my_model --output_dir models/caloric/1 --framework tf

In [38]:
!python -m tf2onnx.convert --saved-model models/caloric/1/my_model --output model.onnx

/usr/lib64/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-09-23 09:27:39,282 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-09-23 09:27:39,408 - INFO - Signatures found in model: [serve,serving_default].
2024-09-23 09:27:39,408 - WARNING - '--signature_def' not specified, using first signature: serve
2024-09-23 09:27:39,408 - INFO - Output names: ['output_0']
I0000 00:00:1727083659.410535    7518 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1727083659.472756    7518 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-09-23 09:27:39,479 - INFO - Using tensorflow=2.17.0, onnx=1.16.2, tf2onnx=1.16.1/15c810
2024-09-23 09:27:39,479 - INFO - Using opset <onnx, 15>
2024-09-23 09:27:39,482 